# Importing the required libraries

In [1]:
!pip install nltk
!pip install pgeocode
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import pandas as pd
import numpy as np
import re
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pgeocode
import plotly.express as px

pd.set_option('display.max_columns', 1000)
pd.set_option('display.float_format', str)

In [3]:
df = pd.read_csv("data.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (0,1,2,3,4,5,7,8,10,11,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,33,34,35,36,37,38,42) have mixed types.Specify dtype option on import or set low_memory=False.



In [4]:
df.head(10)

,Agency Name,Parent Contract ID,Contract ID,Publish Date,Start Date,End Date,Value,Amendment Date,Amendment Start Date,Amendments Value,Description,Agency Ref ID,UNSPSC Code,UNSPSC Title,Procurement Method,ATM ID,SON ID,Panel Arrangement,Confidentiality Contract Flag,Confidentiality Contract Reason,Confidentiality Outputs Flag,Confidentiality Outputs Reason,Consultancy Flag,Consultancy Reason,Amendment Reason,Supplier Name,Supplier Address,Supplier Suburb,Supplier Postcode,Supplier State,Supplier Country,Supplier ABN Exempt,Supplier ABN,Contact Name,Contact Phone,Branch,Division,Office Postcode,Applicable Publish Date,Applicable FY Year,Applicable Value,Duration Years,Applicable Start Date
0,Department of Defence,NaN,CN3620990,20/8/19,1/5/19,31/5/19,16163.4,NaN,NaN,nan,Fitness Equipment,4501071435,49200000.0,Fitness equipment,Limited tender,NaN,NaN,NaN,No,NaN,No,NaN,No,NaN,NaN,BLP TRAINING & SERVICES PTY LTD,NaN,BEENLEIGH,4207,QLD,AUSTRALIA,No,40079623340.0,Army Forces Command,0,Army Forces Command,ARMY HEADQUARTERS,2021,20/8/19,2020.0,16163.0,0.0,1/5/19
1,Australian Bureau of Statistics,CN3602517,CN3602517-A1,21/6/19,1/7/18,30/6/23,1964000.0,25/5/20,25/5/20,648000.0,Software Maintenance,ABS2018.450,81112200.0,Software maintenance and support,Open tender,NaN,SON3520963,Yes,No,NaN,No,NaN,No,NaN,Variation,IBM Australia Limited,NaN,Pennant Hills,1715,NSW,AUSTRALIA,No,79000024733.0,"Director, procurement",NaN,NaN,NaN,2617,25/5/20,2020.0,648000.0,3.0,25/5/20
2,Services Australia,NaN,CN3671573,7/4/20,19/3/20,27/3/20,155997.6,NaN,NaN,nan,Training Services,D296030217,86000000.0,Education and Training Services,Open tender,NaN,SON3393682,Yes,No,NaN,No,NaN,No,NaN,NaN,Benestar Group Pty Ltd,PO Box Q1436,QVB Post Office Sydney,1230,NSW,Australia,No,61003536472.0,PURCHASING HELPDESK,1800 906 144,DHS Branch,DHS Division,2900,7/4/20,2020.0,155998.0,0.0,19/3/20
3,Department of Health,NaN,CN3648860,19/12/19,2/1/20,30/6/20,169542.12,NaN,NaN,nan,Labour Hire,4500135727,80111700.0,Personnel recruitment,Open tender,RFT20000551,SON3403954,Yes,No,NaN,No,NaN,No,NaN,NaN,PEOPLEBANK AUSTRALIA LTD,NaN,NORTH SYDNEY,2060,NSW,Australia,No,42003995748.0,BMU,02 6289 4977,NaN,Cancer Hearing and Program Support DIV,2606,19/12/19,2020.0,169542.0,1.0,2/1/20
4,Administrative Appeals Tribunal,CN3591066,CN3591066-A1,13/5/19,27/2/19,31/7/19,13263.55,25/7/19,1/6/19,13263.55,Labour hire contractor,4500000293,80111600.0,Temporary personnel services,Limited tender,NaN,NaN,NaN,No,NaN,No,NaN,No,NaN,Extension date,LAUNCH RECRUITMENT PTY LTD,LEVEL 10 1 CASTLEREAGH STREET,SYDNEY,2000,NSW,AUSTRALIA,No,54119140840.0,PROCUREMENT OFFICER,(03)9454 6100,Administrative Appeals Tribunal Administrative...,AAT Administrative Appeals Tribunal,4000,25/7/19,2020.0,13264.0,0.0,1/6/19
5,Administrative Appeals Tribunal,NaN,CN3606106,1/7/19,28/6/19,30/6/20,78944.8,NaN,NaN,nan,Avoka software Licensing ans Support renewal,4500000425,43230000.0,Software,Prequalified tender,NaN,NaN,NaN,No,NaN,No,NaN,No,NaN,NaN,AXIENT PTY LTD,PO BOX 119,CROWS NEST,2065,NSW,Australia,No,14090510848.0,PROCUREMENT OFFICER,(03)9454 6100,Administrative Appeals Tribunal Administrative...,AAT Administrative Appeals Tribunal,4000,1/7/19,2020.0,78945.0,1.0,28/6/19
6,Administrative Appeals Tribunal,NaN,CN3661660,24/2/20,11/2/20,31/12/21,54214.11,NaN,NaN,nan,Liberty Recording software maintenance 134 Rec...,4500000561,43230000.0,Software,Open tender,NaN,NaN,NaN,No,NaN,No,NaN,No,NaN,NaN,EPIQ AUSTRALIA PTY LTD,"190 LEVEL 4, QUEEN STREET",MELBOURNE,3000,VIC,Australia,No,85120213381.0,PROCUREMENT OFFICER,(03)9454 6100,Administrative Appeals Tribunal Administrative...,AAT Administrative Appeals Tribunal,4000,24/2/20,2020.0,54214.0,2.0,11/2/20
7,Administrative Appeals Tribunal,NaN,CN3669641,31/3/20,19/3/20,18/3/21,52546.74,NaN,NaN,nan,Cisco Licences,4500000606,43230000.0,Software,Limited tender,NaN,NaN,NaN,No,NaN,No,NaN,No,NaN,NaN,DATACOM SYSTEMS (NSW) PTY LTD,PO BOX 2076,NORTH RYDE,1670,NSW,Australia,No,61055506968.0,PROCUREMENT OF

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292903 entries, 0 to 292902
Data columns (total 43 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Agency Name                      81174 non-null  object 
 1   Parent Contract ID               15920 non-null  object 
 2   Contract ID                      81174 non-null  object 
 3   Publish Date                     81174 non-null  object 
 4   Start Date                       81174 non-null  object 
 5   End Date                         81174 non-null  object 
 6   Value                            81174 non-null  float64
 7   Amendment Date                   15920 non-null  object 
 8   Amendment Start Date             15990 non-null  object 
 9   Amendments Value                 15928 non-null  float64
 10  Description                      81174 non-null  object 
 11  Agency Ref ID                    79716 non-null  object 
 12  UNSPSC Code     

In [6]:
dfNotNa = df.loc[df["Agency Name"].notna(), :]

In [7]:
dfNotNa.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81174 entries, 0 to 81173
Data columns (total 43 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Agency Name                      81174 non-null  object 
 1   Parent Contract ID               15920 non-null  object 
 2   Contract ID                      81174 non-null  object 
 3   Publish Date                     81174 non-null  object 
 4   Start Date                       81174 non-null  object 
 5   End Date                         81174 non-null  object 
 6   Value                            81174 non-null  float64
 7   Amendment Date                   15920 non-null  object 
 8   Amendment Start Date             15990 non-null  object 
 9   Amendments Value                 15928 non-null  float64
 10  Description                      81174 non-null  object 
 11  Agency Ref ID                    79716 non-null  object 
 12  UNSPSC Code       

In [8]:
dfNotNa["Parent Contract ID"].fillna(dfNotNa["Contract ID"], inplace=True)
dfNotNa.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81174 entries, 0 to 81173
Data columns (total 43 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Agency Name                      81174 non-null  object 
 1   Parent Contract ID               81174 non-null  object 
 2   Contract ID                      81174 non-null  object 
 3   Publish Date                     81174 non-null  object 
 4   Start Date                       81174 non-null  object 
 5   End Date                         81174 non-null  object 
 6   Value                            81174 non-null  float64
 7   Amendment Date                   15920 non-null  object 
 8   Amendment Start Date             15990 non-null  object 
 9   Amendments Value                 15928 non-null  float64
 10  Description                      81174 non-null  object 
 11  Agency Ref ID                    79716 non-null  object 
 12  UNSPSC Code       

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [9]:
for col in dfNotNa:
    print(col +": "+ str(len(df[col].unique())))

Agency Name: 120
Parent Contract ID: 12058
Contract ID: 81175
Publish Date: 1353
Start Date: 1893
End Date: 2035
Value: 55223
Amendment Date: 267
Amendment Start Date: 627
Amendments Value: 11709
Description: 32123
Agency Ref ID: 70751
UNSPSC Code: 526
UNSPSC Title: 526
Procurement Method: 4
ATM ID: 7282
SON ID: 683
Panel Arrangement: 3
Confidentiality Contract Flag: 3
Confidentiality Contract Reason: 30
Confidentiality Outputs Flag: 3
Confidentiality Outputs Reason: 22
Consultancy Flag: 3
Consultancy Reason: 4
Amendment Reason: 5237
Supplier Name: 19617
Supplier Address: 8743
Supplier Suburb: 5644
Supplier Postcode: 2805
Supplier State: 10
Supplier Country: 126
Supplier ABN Exempt: 3
Supplier ABN: 11335
Contact Name: 1785
Contact Phone: 898
Branch: 624
Division: 481
Office Postcode: 149
Applicable Publish Date: 279
Applicable FY Year: 2
Applicable Value: 45409
Duration Years: 36
Applicable Start Date: 1183


In [10]:
words = re.findall(r'\w+', " ".join(list(dfNotNa["UNSPSC Title"].unique())).lower())
newwordlist = [word for word in words if not word in stopwords.words()]
Counter(newwordlist)

Counter({'access': 1,
         'accessories': 25,
         'accommodation': 1,
         'accounting': 1,
         'activities': 1,
         'additives': 2,
         'adhesives': 1,
         'administration': 4,
         'administrative': 2,
         'administrators': 1,
         'advertising': 5,
         'advisory': 2,
         'aeronautical': 1,
         'aerospace': 1,
         'agencies': 1,
         'agents': 1,
         'agricultural': 1,
         'aid': 2,
         'aids': 2,
         'air': 5,
         'aircraft': 10,
         'airfield': 1,
         'airport': 2,
         'alarm': 1,
         'alloys': 1,
         'ammunition': 2,
         'anaesthesia': 1,
         'analysis': 2,
         'animal': 2,
         'animals': 1,
         'anti': 1,
         'apparel': 1,
         'appearance': 1,
         'appliances': 1,
         'application': 1,
         'aquaculture': 2,
         'architectural': 2,
         'archive': 1,
         'arms': 1,
         'art': 2,
         'artist

In [11]:
for i in dfNotNa["Supplier Name"].unique():
  if "Bank" in str(i):
    print(i)
  elif "ANZ" in str(i):
    print(i)

SERVICIOS CENTRALES DE COBRANZA
Content Bank Australia Pty Ltd
PeopleBank Australia Limited
PeopleBank Australia Pty Ltd
ECA INTERNATIONAL (Aust Bank Details)
Elizabeth Anne Cheeseman Deutsche Bank Place
Commonwealth Bank
Maddocks - Trust Bank Account
Rydges South Bank Brisbane
Australia and New Zealand Banking Group Limited
Deutsche Bank AG Australia
Commonwealth Bank of Australia
Reserve Bank of Australia
ANZ
TECH DATA ADVANCED SOLUTIONS (ANZ) LIMITED
PeopleBank
Task Technology PtyLtd ATF Caseware ANZ Trust
ANZSOG
GFK ANZ Pty Ltd
GREGORY MARK TANZER
PeopleBank Pty Ltd
DoFD - Official Departmental Bank Acct
The Commonwealth Bank of Australia
GFK ANZ PTY LTD T/As GFK RETAIL AND TECHNOLOGY AUST
MEDED SUPPLIES ANZ PTY LTD
TECH DATA ADVANCED 
SOLUTIONS (ANZ) LIMITED
THE TRUSTEE FOR THE CASEWARE ANZ TRUST
GOPROTO (ANZ) PTY LTD
CAPRIOLI, EVANDRO VENANZI
SOLIDCAM ANZ
BSI GROUP ANZ PTY LIMITED
ALLYANZ PTY LTD
ANZSOG LTD
Banksia Projects
Banksia Scientific Co Pty Ltd
ANZPAA
ANZ BANK
ALLIANZ WO

https://www.finder.com.au/list-of-banks-in-australia
https://en.wikipedia.org/wiki/List_of_banks_in_Australia

In [12]:
banklistdf = pd.read_csv("banklist.csv")
banklistdf

,Bank,Headquarters,Established,Owned by / Subsidiary of
0,86400 Limited,Sydney,2019,NaN
1,AMP Bank,Sydney,1849,NaN
2,Australia & New Zealand Banking Group (ANZ),Melbourne,1835,NaN
3,Australian Military Bank (previously Australia...,Sydney,1959,NaN
4,Australian Settlements Limited (ASL),Canberra,1993,NaN
...,...,...,...,...
223,The Mutual,NaN,NaN,NaN
224,Your Credit Union,NaN,NaN,NaN
225,myMOVE,NaN,NaN,NaN
226,Lifeplan Australia Friendly Society,NaN,NaN,NaN


In [13]:
bankcreditlist = list(banklistdf.loc[~banklistdf["Bank"].str.lower().str.contains("bank|credit") , "Bank"]
     .str.replace(" Limited", "")
     .str.replace(" Ltd", "")
     .str.replace('\(.*$', "", regex=True)
     .str.replace('\[.*$', "", regex=True))

In [14]:
for eachcomp in [i.lower() for i in bankcreditlist]:
  if dfNotNa.loc[dfNotNa["Supplier Name"].str.lower().str.contains(eachcomp) , :].shape[0] > 0:
     print(eachcomp)


clean energy finance corporation
up
ecu
imb
catalyst
asl
the mac


In [15]:
dfNotNa.loc[dfNotNa["Supplier Name"].str.lower().str.contains("the mac") , :]

,Agency Name,Parent Contract ID,Contract ID,Publish Date,Start Date,End Date,Value,Amendment Date,Amendment Start Date,Amendments Value,Description,Agency Ref ID,UNSPSC Code,UNSPSC Title,Procurement Method,ATM ID,SON ID,Panel Arrangement,Confidentiality Contract Flag,Confidentiality Contract Reason,Confidentiality Outputs Flag,Confidentiality Outputs Reason,Consultancy Flag,Consultancy Reason,Amendment Reason,Supplier Name,Supplier Address,Supplier Suburb,Supplier Postcode,Supplier State,Supplier Country,Supplier ABN Exempt,Supplier ABN,Contact Name,Contact Phone,Branch,Division,Office Postcode,Applicable Publish Date,Applicable FY Year,Applicable Value,Duration Years,Applicable Start Date
58089,Department of Health,CN3631377,CN3631377,1/10/19,27/9/19,30/6/20,850800.06,NaN,NaN,nan,Procure Research Data,4500134773,93131700.0,Health programs,Limited tender,HEALTH/2019-20,NaN,NaN,No,NaN,No,NaN,No,NaN,NaN,The Macfarlane Burnet Institute for Medical Re...,NaN,MELBOURNE,3004,VIC,Australia,No,49007349984.0,BMU,02 6289 1555,NaN,Pharmaceutical Benefits Division,2606,1/10/19,2020.0,850800.0,1.0,27/9/19


In [16]:
finaldata = dfNotNa.loc[dfNotNa["Supplier Name"].str.lower().str.contains("bank|credit|clean energy finance corporation| up |the mac") , [ "Agency Name","Value", "Duration Years"]].groupby("Agency Name").agg(['mean', 
                                                                                                                                                                                                     'median', 
                                                                                                                                                                                                     'sum',
                                                                                                                                                                                                     'min',
                                                                                                                                                                                                     'max',
                                                                                                                                                                                                     'count'])

In [17]:
finaldata = finaldata.loc[finaldata['Value',  'count'] > 1, :]
finaldata.sort_values(by=('Value','mean'),ascending=False)


Value  \
                                                                 mean   
Agency Name                                                             
Department of Defence                               75647951.53090227   
Australian Office of Financial Management          2837015.8333333335   
Department of Home Affairs                          639461.9052307692   
Bureau of Meteorology                              515989.66684210516   
IP Australia                                              486887.7125   
Australian Criminal Intelligence Commission                  483333.2   
Services Australia                                 480814.91653061233   
Department of Health                                410831.8814285714   
Department of the Prime Minister and Cabinet                381163.86   
Department of Veterans' Affairs                     366133.3333333333   
Australian Taxation Office                          323959.6365217391   
Australian Federal Police                          320123.93333333335   
Office of Parliamentary Counsel                              309800.0   
Department of the Treasury                                   297500.0   
Australian Securities and Investments Commission            282573.25   
Future Fund Management Agency                                279950.5   
Australian Electoral Commission                             271597.25   
Australian Skills Quality Authority                 250959.2285714286   
Department of Industry, Science, Energy and Res... 237921.88916666666   
Department of Foreign Affairs and Trade                   236207.5875   
Department of Agriculture, Water and the Enviro...       220919.52625   
Australian Bureau of Statistics                    217305.11111111112   
Department of Education, Skills and Employment      211608.1846153846   
Department of Finance                              168293.65473684212   
Australian Signals Directorate                               137197.5   
Department of Social Services                                119735.0   
Australian Communications and Media Authority             102903.5625   
Department of Infrastructure, Transport, Region...  86592.51354838711   
Clean Energy Regulator                                        60300.0   
Australian Financial Security Authority                      40984.75   
Department of Parliamentary Services                        31378.548   
Australian Prudential Regulation Authority         28339.667999999998   

                                                                       \
                                                               median   
Agency Name                                                             
Department of Defence                                      4229161.58   
Australian Office of Financial Management                   3698750.0   
Department of Home Affairs                                   637175.0   
Bureau of Meteorology                                       487652.22   
IP Australia                                               362854.555   
Australian Criminal Intelligence Commission                  521976.4   
Services Australia                                           343200.0   
Department of Health                                         286374.0   
Department of the Prime Minister and Cabinet                 314890.7   
Department of Veterans' Affairs                              340000.0   
Australian Taxation Office                         290411.08999999997   
Australian Federal Police                                    229835.2   
Office of Parliamentary Counsel                              309800.0   
Department of the Treasury                                   297500.0   
Australian Securities and Investments Commission             198880.0   
Future Fund Management Agency                                279950.5   
Australian Electoral Commission                              79818.75   
Australian Skills Quality Authority             

In [18]:
finaldata.sort_values(by=('Value','max'),ascending=False)

Value  \
                                                                 mean   
Agency Name                                                             
Department of Defence                               75647951.53090227   
Australian Office of Financial Management          2837015.8333333335   
Department of Health                                410831.8814285714   
Services Australia                                 480814.91653061233   
Department of Home Affairs                          639461.9052307692   
Australian Taxation Office                          323959.6365217391   
Bureau of Meteorology                              515989.66684210516   
Department of the Prime Minister and Cabinet                381163.86   
Australian Criminal Intelligence Commission                  483333.2   
Australian Federal Police                          320123.93333333335   
IP Australia                                              486887.7125   
Department of Infrastructure, Transport, Region...  86592.51354838711   
Australian Electoral Commission                             271597.25   
Department of Industry, Science, Energy and Res... 237921.88916666666   
Australian Bureau of Statistics                    217305.11111111112   
Department of Foreign Affairs and Trade                   236207.5875   
Department of Finance                              168293.65473684212   
Australian Securities and Investments Commission            282573.25   
Department of Veterans' Affairs                     366133.3333333333   
Office of Parliamentary Counsel                              309800.0   
Department of the Treasury                                   297500.0   
Future Fund Management Agency                                279950.5   
Department of Agriculture, Water and the Enviro...       220919.52625   
Department of Education, Skills and Employment      211608.1846153846   
Australian Skills Quality Authority                 250959.2285714286   
Department of Social Services                                119735.0   
Australian Signals Directorate                               137197.5   
Australian Communications and Media Authority             102903.5625   
Clean Energy Regulator                                        60300.0   
Australian Prudential Regulation Authority         28339.667999999998   
Australian Financial Security Authority                      40984.75   
Department of Parliamentary Services                        31378.548   

                                                                       \
                                                               median   
Agency Name                                                             
Department of Defence                                      4229161.58   
Australian Office of Financial Management                   3698750.0   
Department of Health                                         286374.0   
Services Australia                                           343200.0   
Department of Home Affairs                                   637175.0   
Australian Taxation Office                         290411.08999999997   
Bureau of Meteorology                                       487652.22   
Department of the Prime Minister and Cabinet                 314890.7   
Australian Criminal Intelligence Commission                  521976.4   
Australian Federal Police                                    229835.2   
IP Australia                                               362854.555   
Department of Infrastructure, Transport, Region...            38494.5   
Australian Electoral Commission                              79818.75   
Department of Industry, Science, Energy and Res...           148280.0   
Australian Bureau of Statistics                              116400.0   
Department of Foreign Affairs and Trade                      169708.0   
Department of Finance                                        135291.0   
Australian Securities and Investments Commission

In [19]:
dfNotNa.loc[dfNotNa["Agency Name"] == "Department of Defence",["Agency Name", "Supplier Address", 
         "Supplier Suburb", 
         "Supplier Postcode" , 
         "Supplier State",
         "Supplier Country"]]

,Agency Name,Supplier Address,Supplier Suburb,Supplier Postcode,Supplier State,Supplier Country
0,Department of Defence,NaN,BEENLEIGH,4207,QLD,AUSTRALIA
18539,Department of Defence,NaN,JOHNSTON,831,NT,AUSTRALIA
18540,Department of Defence,NaN,LANE COVE NORTH,2066,NSW,AUSTRALIA
18541,Department of Defence,NaN,THEBARTON,5031,SA,AUSTRALIA
18542,Department of Defence,NaN,LANE COVE WEST,2066,NSW,AUSTRALIA
...,...,...,...,...,...,...
51357,Department of Defence,NaN,ALBURY,2640,NSW,AUSTRALIA
51358,Department of Defence,NaN,NORTH SYDNEY,2060,NSW,AUSTRALIA
51359,Department of Defence,NaN,GRIFFITH,2603,ACT,AUSTRALIA
51360,Department of Defence,NaN,ARCHERFIELD,4108,QLD,AUSTRALIA


In [20]:
finaldata.sort_values(by=('Value','max'),ascending=False).index

Index(['Department of Defence', 'Australian Office of Financial Management',
       'Department of Health', 'Services Australia',
       'Department of Home Affairs', 'Australian Taxation Office',
       'Bureau of Meteorology', 'Department of the Prime Minister and Cabinet',
       'Australian Criminal Intelligence Commission',
       'Australian Federal Police', 'IP Australia',
       'Department of Infrastructure, Transport, Regional Development and Communications',
       'Australian Electoral Commission',
       'Department of Industry, Science, Energy and Resources',
       'Australian Bureau of Statistics',
       'Department of Foreign Affairs and Trade', 'Department of Finance',
       'Australian Securities and Investments Commission',
       'Department of Veterans' Affairs', 'Office of Parliamentary Counsel',
       'Department of the Treasury', 'Future Fund Management Agency',
       'Department of Agriculture, Water and the Environment',
       'Department of Education, S

In [21]:
postCodeList = []
for eachCustomer in list(finaldata.sort_values(by=('Value','max'),ascending=False).index):
  postCodeList.append(dfNotNa.loc[(dfNotNa["Agency Name"] == eachCustomer) & 
              (dfNotNa["Supplier Country"] == "AUSTRALIA"),"Supplier Postcode"].values)

In [22]:
flat_list = [item for sublist in postCodeList for item in sublist]
print(flat_list)

['4207', '831', '2066', '5031', '2066', '2600', '2603', '2617', '2035', '2600', '2601', '2609', '3205', '5031', '3027', '3153', '2035', '3004', '2294', '2060', '4106', '2609', '3000', '2778', '3170', '2107', '5013', '2113', '4165', '2612', '2138', '6065', '3170', '2232', '2060', '2600', '3141', '2612', '4210', '3180', '2113', '2617', '3004', '3175', '2600', '2150', '2612', '3004', '5013', '2609', '2620', '3010', '2164', '2000', '2019', '4870', '2000', '3010', '5039', '2015', '2010', '6105', '6021', '3803', '6025', '3006', '5111', '2116', '4008', '2000', '3216', '6150', '2127', '4072', '4106', '3000', '2000', '2607', '2020', '2612', '2609', '4055', '2294', '5031', '2601', '2612', '2617', '2601', '2601', '4811', '5000', '6162', '2011', '2609', '4567', '5252', '3350', '5010', '3685', '2611', '4218', '3202', '2000', '2601', '4811', '3008', '2617', '2000', '3195', '5106', '3000', '2000', '3067', '5106', '2068', '4810', '6155', '5031', '3207', '5000', '2600', '3136', '2164', '3004', '2324', 

In [23]:
postcodelistlonglat = []
nomi = pgeocode.Nominatim('AU')
for eachpost in set(flat_list):
  print(eachpost)
  postcodelistlonglat.append(nomi.query_postal_code(eachpost))

4562
8009
3195
1885
3093
850
3934
828
3057
3016
3363
2282
4173
2035
5211
3898
3174
2323
1232
4558
5252
5106
6060
2228
4210
2284
2519
2779
4132
2223
2527
4878
2125
6532
4306
2102
5031
3000
2758
5433
6069
7009
2460
3132
909
1235
6109
3666
2787
8006
4895
2608
3158
801
6817
7000
6031
2128
3066
3136
3002
3505
3083
871
2778
5044
6012
2602
6850
6951
1871
0
2624
5019
6910
2320
4096
3074
2290
4073
2300
4157
5223
3173
3280
3044
4075
2520
2899
3353
2044
2751
4352
5461
2479
3143
4561
1715
5026
5167
3472
6076
4874
4740
2646
4151
2760
4076
5240
3800
4072
2066
2471
3825
812
2179
5069
2540
4892
2765
3931
3043
3658
2012
5018
2130
6160
3818
4037
2135
2153
2578
3178
4817
2840
2062
4154
2880
6017
3806
2539
2264
2109
7050
2064
3215
2095
4508
3868
2096
5155
3026
1234
5109
6081
2303
3435
3135
5606
3103
2141
2560
4700
3807
2046
4205
2154
6966
4020
2002
3920
3601
18983
4053
5950
6143
4560
1335
7190
4122
2444
3133
4868
1660
7179
3076
2525
3765
6108
3047
7253
6056
3130
2722
2486
4006
3005
3121
6019
3212
2099
456

In [24]:
longlatdf = pd.DataFrame(postcodelistlonglat)
longlatdf.head()

,postal_code,country_code,place_name,state_name,state_code,county_name,county_code,community_name,community_code,latitude,longitude,accuracy
0,4562,AU,"Doonan, Belli Park, Weyba Downs, Eerwah Vale, ...",Queensland,QLD,SUNSHINE MT ISA,nan,nan,nan,-26.4502,152.95918333333333,4.0
0,8009,AU,Flinders Lane,Victoria,VIC,MELBOURNE CITY,nan,nan,nan,-37.8153,144.9756,1.0
0,3195,AU,"Waterways, Parkdale, Aspendale Gardens, Braesi...",Victoria,VIC,DANDENONG,nan,nan,nan,-38.0139,145.10508333333334,4.0
0,1885,AU,Bankstown,New South Wales,NSW,BANKSTOWN,nan,nan,nan,-33.9167,151.0333,4.0
0,3093,AU,Lower Plenty,Victoria,VIC,PRESTON,nan,nan,nan,-37.7295,145.1181,4.0


In [25]:
fig = px.scatter_geo(longlatdf,lat='latitude',lon='longitude', hover_name="place_name")
fig.update_layout(title = 'World map', title_x=0.5)
fig.show()

In [27]:
import plotly.express as px
import geopandas as gpd
px.set_mapbox_access_token("pk.eyJ1IjoicmFzaGJpcmtvaGxpIiwiYSI6ImNrcWs4MnU1djAzZHIzMHRndnFodTRvNjgifQ.r2Wo44z-LyC8PO14JjaeXg")
fig = px.scatter_mapbox(longlatdf,lat='latitude',lon='longitude', hover_name="place_name",
                        zoom=1)
fig.show()